This version specifies a random portion of missingness in the features. It is identical with missing_cleanml-2-percentage. These two notebooks are used to run datasets in parallel. Airbnb in particular takes a long time to run.

In [1]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import utils
import pickle
from collections import defaultdict
import numpy as np
from collections import OrderedDict
from sklearn.metrics import f1_score 
import pandas as pd
from sklearn.metrics import f1_score 

/mnt/ds3lab-scratch/raox/gatech_data_cleaning/ICDE2021-revision/nacl36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/mnt/ds3lab-scratch/raox/gatech_data_cleaning/ICDE2021-revision/nacl36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import sys
sys.setrecursionlimit(10**6)

In [3]:
import math
import matplotlib.pyplot as plt 
from sklearn.datasets import load_digits
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pylab
import sys
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
import keras
import time
import pandas as pd
import json
import operator

Using TensorFlow backend.


In [4]:
sys.path.append("../NaCL/code/")
import utils
import LR2NB_GP
sys.path.append("../NaCL/notebooks/")
from dataload import *

In [5]:
from preprocess_mv import preprocess_data

/mnt/ds3lab-scratch/raox/gatech_data_cleaning/ICDE2021-revision/nacl36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [6]:
%load_ext autoreload
%autoreload 2

# Loading Data

In [7]:
seed = 3659

In [8]:
data_dir = "data-robustml-mv"

In [9]:
save_dir = "result-robustml"
model_name ='nacl'

In [10]:
accuracy = lambda x,y: np.mean(x == y)
f1 = lambda x,y: f1_score(x,y.flatten(), average = "weighted")



In [11]:
dataset = 'Credit'

In [12]:
jobs = get_jobs_mv(data_dir, dataset = [dataset])
print("Number of jobs:", len(jobs))

Number of jobs: 20


In [13]:
jobs

[['data-robustml-mv', 'Credit', 'missing_values', 'split_3462'],
 ['data-robustml-mv', 'Credit', 'missing_values', 'split_3562'],
 ['data-robustml-mv', 'Credit', 'missing_values', 'split_905'],
 ['data-robustml-mv', 'Credit', 'missing_values', 'split_5056'],
 ['data-robustml-mv', 'Credit', 'missing_values', 'split_5374'],
 ['data-robustml-mv', 'Credit', 'missing_values', 'split_144'],
 ['data-robustml-mv', 'Credit', 'missing_values', 'split_4225'],
 ['data-robustml-mv', 'Credit', 'missing_values', 'split_7813'],
 ['data-robustml-mv', 'Credit', 'missing_values', 'split_2516'],
 ['data-robustml-mv', 'Credit', 'missing_values', 'split_2895'],
 ['data-robustml-mv', 'Credit', 'missing_values', 'split_4764'],
 ['data-robustml-mv', 'Credit', 'missing_values', 'split_7751'],
 ['data-robustml-mv', 'Credit', 'missing_values', 'split_2962'],
 ['data-robustml-mv', 'Credit', 'missing_values', 'split_8093'],
 ['data-robustml-mv', 'Credit', 'missing_values', 'split_5192'],
 ['data-robustml-mv', 'Cred

In [14]:
accuracy = lambda x,y: np.mean(x == y)
f1 = lambda x,y: f1_score(x,y.flatten(), average = "weighted")



In [15]:
with open('/mnt/ds3lab-scratch/raox/gatech_data_cleaning/ICDE2021-revision/CleanML/result/{}_result.json'.format(dataset)) as json_file:
    results = json.load(json_file)

In [16]:
%%time 

for job in jobs:
    
    print("Processing", job)
    print('--------------')
    dataset = job[1]

    dirty_X_train = pd.read_csv(os.path.join(*job, "dirty_X_train.csv"))
    dirty_y_train = pd.read_csv(os.path.join(*job, "dirty_y_train.csv"))
    dirty_X_test = pd.read_csv(os.path.join(*job, "dirty_X_test.csv"))
    dirty_y_test = pd.read_csv(os.path.join(*job, "dirty_y_test.csv"))

    dirty_X_train = dirty_X_train.fillna(0)
    dirty_X_test = dirty_X_test.fillna(0)
    X_train, y_train, X_test, y_test, mappings, mappings_dict = preprocess_data(dataset, 
                                                             dirty_X_train, dirty_y_train, 
                                                             [dirty_X_test], [dirty_y_test])
    X_test = X_test[0]
    y_test = y_test[0]
    dirty_X_test = pd.read_csv(os.path.join(*job, "dirty_X_test.csv"))
    null_data = dirty_X_test[dirty_X_test.isnull().any(axis=1)]

    missing = [[]] * dirty_X_test.shape[0]
    for i, row in null_data.iterrows():
        # append the original index of the missing feature
        cols = null_data.columns.get_values()[np.where(pd.isnull(row))]

        idx = [mappings_dict[col][0] for col in cols]
        missing[i] = idx
    missing = np.array(missing)

    output = defaultdict(lambda: defaultdict(list))

    output['train']['nacl'] = (X_train, y_train)
    output['test']['nacl'] = (X_test, y_test)
    output['missing_test'] = missing
    output['mappings1hot_dict'] = mappings_dict
    output['mappings1hot_list'] = mappings
    pickle.dump(dict(output), open(os.path.join(*job,"data.p"), 'wb'))


    max_acc = 0

    for method in ['delete', 'impute_holoclean', 'impute_mean_mode', 'impute_mean_dummy', 
                   'impute_median_mode', 'impute_median_dummy', 'impute_mode_mode', 'impute_mode_dummy']:
        #print('Titanic/v{}/missing_values/{}/logistic_regression/3659'.\
        #           format(split_seed, method))
        split_seed = job[-1].split("_")[1]
        a =  results['{}/v{}/missing_values/{}/logistic_regression/3659'.\
                   format(dataset, split_seed, method)].copy()
        #print(a)
        del a['best_params']
        del a['train_acc']
        del a['val_acc']
        max_key = max(a.items(), key=operator.itemgetter(1))[0]

        if a[max_key] > max_acc:
            best_params_lr = results['{}/v{}/missing_values/{}/logistic_regression/3659'.\
                   format(dataset, split_seed, method)]['best_params']
            max_acc = a[max_key]

    clf = LogisticRegression(solver='lbfgs', 
    verbose=True, 
    C = best_params_lr['C'],
    max_iter=2000, n_jobs=-1).fit(X_train, y_train)
    lr_pred = clf.predict(X_test)
    lr_train_acc = accuracy(clf.predict(X_train), y_train)
    lr_test_acc = accuracy(lr_pred, y_test)
    lr_test_f1 = f1(lr_pred, y_test)

    print("LR Train Accuracy: {} \nLR Test  Accuracy: {}".format(lr_train_acc, lr_test_acc))

    W = np.hstack((clf.intercept_[:,None], clf.coef_))[0]
    nacl = LR2NB_GP.LR2NB(W)
    nacl.setObj(X_train, y_train)

    # Using Mosek Solver
    nacl.solve(verbose=1)

    # Sanity Check to compare NaCL results with Logistic Regression
    assert(1.0 == np.average( nacl.classify(X_test)  == clf.predict(X_test )))
    assert(1.0 == np.average( nacl.classify(X_train) == clf.predict(X_train)))
    nacl_pred = nacl.classify(X_test)
    nacl_train_acc = accuracy(nacl.classify(X_train), y_train)
    nacl_test_acc = accuracy(nacl_pred, y_test)
    #print(nacl_pred)
    #print(y_test)
    nacl_test_f1 = f1(nacl_pred[0], y_test)

    print("NaCL Train Accuracy: {} \nNaCL Test  Accuracy: {}".format(nacl_train_acc, nacl_test_acc))

    NB = BernoulliNB().fit(X_train, y_train)
    NB_pred = NB.predict(X_test)
    nb_train_acc = accuracy(NB.predict(X_train), y_train)
    nb_test_acc = accuracy(NB_pred, y_test)
    nb_test_f1 = f1(NB_pred, y_test)

    print("NB Train Accuracy: {} \nNB Test  Accuracy: {}".format(nb_train_acc, nb_test_acc))

    # Make an example observations with some missing features

    # setting the dimensions to be zero
    missing_mask = np.zeros(X_test.shape) 


    miss = missing
    for i in range(len(missing_mask)):
        for j in range(len(miss[i])):
            missing_mask[i][mappings[miss[i][j]]] = 1

    a = nacl.predict_proba(X_test, missing = missing_mask)[:,1]
    a = np.where(a > 0.5, 1, 0)

    test_acc = accuracy(y_test, a)
    test_f1 = f1(y_test, a)
    print('nacl test acc with missing:', test_acc)
    print('nacl test f1 with missing:', test_f1) # true, pred


    result_dict = {}

    result_dict["best_params"] = best_params_lr


    result_dict["{}_test_f1".format('nacl_missing')] = test_f1
    result_dict["{}_test_acc".format('nacl_missing')] = test_acc

    result_dict["{}_test_f1".format('nacl')] = nacl_test_f1
    result_dict["{}_test_acc".format('nacl')] = nacl_test_acc
    result_dict["{}_train_acc".format('nacl')] = nacl_train_acc

    result_dict["{}_test_f1".format('nb')] = nb_test_f1
    result_dict["{}_test_acc".format('nb')] = nb_test_acc
    result_dict["{}_train_acc".format('nb')] = nb_train_acc

    result_dict["{}_test_f1".format('lr')] = lr_test_f1
    result_dict["{}_test_acc".format('lr')] = lr_test_acc
    result_dict["{}_train_acc".format('lr')] = lr_train_acc

    print(result_dict)
    save_result_mv(job, seed, result_dict, model_name, save_dir)





Processing ['data-robustml-mv', 'Credit', 'missing_values', 'split_3462']
--------------
dataset: {'data_dir': 'Credit', 'error_types': ['outliers', 'missing_values'], 'label': 'SeriousDlqin2yrs', 'categorical_variables': ['SeriousDlqin2yrs'], 'ml_task': 'classification', 'class_imbalance': True}
columns to encode: []


/mnt/ds3lab-scratch/raox/gatech_data_cleaning/ICDE2021-revision/nacl36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.4s finished


LR Train Accuracy: 0.7229254571026723 
LR Test  Accuracy: 0.742
Using solver 'cvxopt'
 for 42 free variables
  in 44 posynomial inequalities.
Solving took 0.112 seconds.
NaCL Train Accuracy: 0.7229254571026723 
NaCL Test  Accuracy: 0.742
NB Train Accuracy: 0.7630098452883263 
NB Test  Accuracy: 0.7964444444444444
nacl test acc with missing: 0.7473333333333333
nacl test f1 with missing: 0.8074443904158991
{'best_params': {'C': 10594.381544832824}, 'nacl_missing_test_f1': 0.8074443904158991, 'nacl_missing_test_acc': 0.7473333333333333, 'nacl_test_f1': 0.6803515993516512, 'nacl_test_acc': 0.742, 'nacl_train_acc': 0.7229254571026723, 'nb_test_f1': 0.7516192945978029, 'nb_test_acc': 0.7964444444444444, 'nb_train_acc': 0.7630098452883263, 'lr_test_f1': 0.6803515993516512, 'lr_test_acc': 0.742, 'lr_train_acc': 0.7229254571026723}
Processing ['data-robustml-mv', 'Credit', 'missing_values', 'split_3562']
--------------
dataset: {'data_dir': 'Credit', 'error_types': ['outliers', 'missing_values'

/mnt/ds3lab-scratch/raox/gatech_data_cleaning/ICDE2021-revision/nacl36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


columns to encode: []


...

columns to encode: []


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


LR Train Accuracy: 0.6937134502923976 
LR Test  Accuracy: 0.7075555555555556
Using solver 'cvxopt'
 for 42 free variables
  in 44 posynomial inequalities.
Solving took 0.14 seconds.
NaCL Train Accuracy: 0.6937134502923976 
NaCL Test  Accuracy: 0.7075555555555556
NB Train Accuracy: 0.6980994152046783 
NB Test  Accuracy: 0.8066666666666666
nacl test acc with missing: 0.7153333333333334
nacl test f1 with missing: 0.7867039332752519
{'best_params': {'C': 1286.4014641512479}, 'nacl_missing_test_f1': 0.7867039332752519, 'nacl_missing_test_acc': 0.7153333333333334, 'nacl_test_f1': 0.6339660638155262, 'nacl_test_acc': 0.7075555555555556, 'nacl_train_acc': 0.6937134502923976, 'nb_test_f1': 0.7643389325554937, 'nb_test_acc': 0.8066666666666666, 'nb_train_acc': 0.6980994152046783, 'lr_test_f1': 0.6339660638155262, 'lr_test_acc': 0.7075555555555556, 'lr_train_acc': 0.6937134502923976}
Processing ['data-robustml-mv', 'Credit', 'missing_values', 'split_235']
--------------
dataset: {'data_dir': 'Cre

/mnt/ds3lab-scratch/raox/gatech_data_cleaning/ICDE2021-revision/nacl36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


LR Train Accuracy: 0.7 
LR Test  Accuracy: 0.6842222222222222
Using solver 'cvxopt'
 for 42 free variables
  in 44 posynomial inequalities.
Solving took 0.184 seconds.
NaCL Train Accuracy: 0.7 
NaCL Test  Accuracy: 0.6842222222222222
NB Train Accuracy: 0.723943661971831 
NB Test  Accuracy: 0.824
nacl test acc with missing: 0.7024444444444444
nacl test f1 with missing: 0.775662863882907
{'best_params': {'C': 1286.4014641512479}, 'nacl_missing_test_f1': 0.775662863882907, 'nacl_missing_test_acc': 0.7024444444444444, 'nacl_test_f1': 0.6060303599510445, 'nacl_test_acc': 0.6842222222222222, 'nacl_train_acc': 0.7, 'nb_test_f1': 0.7892721333084979, 'nb_test_acc': 0.824, 'nb_train_acc': 0.723943661971831, 'lr_test_f1': 0.6060303599510445, 'lr_test_acc': 0.6842222222222222, 'lr_train_acc': 0.7}
CPU times: user 17.3 s, sys: 38.2 s, total: 55.5 s
Wall time: 51 s
